In [9]:
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter, MaxNLocator
from scipy.interpolate import griddata

import numpy as np

%matplotlib qt

In [129]:
base = "D:/Ayush/Mech-Project/Vibration-Analysis/Data/"

In [145]:
df1 = pd.read_csv(base+"Fixed-Fixed.csv")
df2 = pd.read_csv(base+"Fixed-Fixed-Test.csv")

In [161]:
df = pd.concat([df1, df2])

In [162]:
df.head()

,depth,l1,l2,Mode,Frequency,Max_Deflection,Average_Deflection
0,0.0,0,0,1,52.976,18.084,9.0351
1,0.0,0,0,2,145.920,17.293,9.3771
2,0.0,0,0,3,285.990,17.430,9.5803
3,0.0,0,0,4,323.400,28.366,10.8720
4,0.0,0,0,5,472.810,17.537,9.7190


# Data

In [164]:
def get_xyz(df, indexer = "Mode", index_val=None, param1 = "l1", param2 = "l2", param3 = "Frequency"):
    """
    Input:
    - indexer: Conditional Varaible to Split the data
    - index_val: Value of the Conditional Variable
    - param1: X Variable
    - param2: Y Variable
    - param3: Z Variable
    
    Output:
    X, Y, Z cordiantes for Plotting
    """
    idx = df[indexer] == index_val
    view = np.vstack((df[[param1, param2, param3]][idx], df[[param2, param1, param3]][idx]))
    return view[:, 0], view[:, 1], view[:, 2]

In [168]:
X, Y, Z = get_xyz(df = df, indexer="Mode", index_val=2)

# Surface Plots with Triangulization

Ref: https://stackoverflow.com/questions/44244297/plot-trisurf-of-matplotlib-2-0-2

In [166]:
%matplotlib qt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_trisurf(X, Y, Z, 
                       cmap = cm.jet, 
                       linewidth = 2,
                       alpha = 0.7)
fig.colorbar(surf)

ax.xaxis.set_major_locator(MaxNLocator(10))
ax.yaxis.set_major_locator(MaxNLocator(10))
ax.zaxis.set_major_locator(MaxNLocator(10))

fig.tight_layout()
plt.show()

# Surface Plots with Interpolation

Ref: https://stackoverflow.com/questions/9170838/surface-plots-in-matplotlib

In [207]:
X, Y, Z = get_xyz(df = df, indexer="Mode", index_val=6, param3="Average_Deflection")

%matplotlib qt

x1 = np.linspace(0, 1000, 200)
y1 = np.linspace(0, 1000, 200)

x2, y2 = np.meshgrid(x1, y1)
z2 = griddata((X, Y), Z, (x2, y2), method='cubic', fill_value=Z.mean())


no_nan = ~np.isnan(z2) # To Check the utility of the Interpolation

fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(x2, y2, z2, 
                       rstride = 2, 
                       cstride = 2, 
                       cmap = cm.brg, 
                       linewidth = 0,
                       alpha = 0.7,
                       antialiased = False)


fig.colorbar(surf)

ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

plt.show()